In [1]:
import pandas as pd
import numpy as np

### Defining function for data cleaning

In [2]:
def clean_data(df):
    missing_values = df.isnull().sum()
    mv_ratio = missing_values[missing_values.gt(0)]/len(df)
    list_mv_50 = mv_ratio.loc[mv_ratio.gt(0.5)].index
    
    train_01 = df.drop( columns = list_mv_50).copy()
    
    mv_01 = train_01.isnull().sum()
    list_cols = mv_01.loc[mv_01.gt(0)].index
    
    for col in list_cols:
        train_01[col] = train_01[col].fillna(0)
    
    return train_01

In [3]:
test = pd.read_csv('../Data/nfl_test.csv')

/Users/ulrikeanklam/opt/anaconda3/envs/matrix/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,226,227,231,232,233,238,240,241) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
test = clean_data(test)

In [5]:
# export cleaned test df
# test = test.to_csv('../Data/test_clean.csv', index = False)

### Defining function for data preprocessing

In [11]:
def process_data(df):
    df['succesful_play'] = (np.where((df['play_type'] == 'pass')
                                        & (df['yards_gained']>= 6.3), 1,
                                   np.where((df['play_type'] == 'run') 
                                            & (df['yards_gained'] >= 4.4), 1, 0)))
    
    df['game_setting'] = df['home_team'].str.cat(df['away_team'])
    dummies = pd.get_dummies(df['game_setting'])
    df = df.join(dummies)
    
    df['posteam_is_home'] = (np.where(df['posteam_type'] == 'home', 1 , 0))
    
    playtype = pd.get_dummies(df['play_type'], prefix= 'pt', drop_first = True )
    df = df.join(playtype)
    
    df['game_half'] = np.where(df['game_half'] == 'Half1', 1, 2)
    
    df['side_of_field_is_hometeam'] = np.where(df['side_of_field'] == df['home_team'], 1, 0)
    
    object_cols = ['home_team','away_team' ,'posteam' ,'posteam_type',
               'defteam', 'side_of_field','game_date','time','yrdln', 'desc','play_type', 'game_setting']
    df = df.drop(columns = object_cols)
    return df 

In [12]:
test_ready = process_data(test)

In [14]:
# export processed test df
# export_test = test_ready.to_csv('../Data/test_ready.csv', index = False)